In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Utility Functions
## For search algorithms and board evaluation

### Dependencies

In [ ]:
import chess
import chess.polyglot
from typing import Union

import import_ipynb
from Globals import Globals

importing Jupyter notebook from Globals.ipynb


#### chess.Board.evaluate_move
Calculates the board score after a given move on the current board state. A high score represents a good move and a low score represents a bad move. The evaluation is performed over multiple phases, which account for game overs, endgame tablebase information, attacks, positional changes, castling and promotions.

###### <b><u>Arguments</u></b>
``use_heuristic (bool):``  
Whether or not the heuristic for evaluating the chess board should be used. Chess problems don't need this heuristic.  

``color (chess.Color):``  
The color (white or black) of the player that is able to make the move.

``is_opponent (bool):``  
Whether or not a potential move of the opponent is evaluated (used in minimax and alpha-beta searches).

``prev_score (int):``  
The score of the board state before the move is made. The move's score is added to this value to calculate the total score.

``iteration (int):``  
The current iteration (depth) of the search.

``move (chess.Move):``  
The move that should be evaluated.

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.

###### <b><u>Returns <i>(int, bool)</i></u></b>
- The total score (previous board score + move score).
- Whether or not the endgame library was used to find the move.

In [ ]:
def evaluate_move(
    self,
    use_heuristic: bool,
    color: chess.Color,
    is_opponent: bool,
    prev_score: int,
    iteration: int,
    move: chess.Move,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
        chess.gaviota.PythonTablebase]
) -> (int, bool):

    is_opponent_multiplier = -1 if is_opponent else 1

    # Handle ending the game (draw = neutral, win = highest score, loss = lowest score)
    game_ended, winner = self.evaluate_game_end(move)
    if game_ended:
        if winner is None:
            draw_score = -Globals.EVALUATION_SCORE_DRAW if self.is_winning(color) \
                    else Globals.EVALUATION_SCORE_DRAW
            return draw_score * is_opponent_multiplier, False
        return (Globals.EVALUATION_SCORE_CHECKMATE - iteration) * is_opponent_multiplier \
                * (1 if winner == color else -1), False

    # If the game hasn't ended and we don't use a chess board heuristic,
    # the score doesn't matter
    if not use_heuristic:
        return 0, False

    # Handle endgame tablebase
    if endgame_tablebase is not None:
        endgame_tablebase_score = self.evaluate_endgame(move, color, endgame_tablebase)
        if endgame_tablebase_score is not None:
            return endgame_tablebase_score * is_opponent_multiplier, True

    # If the game hasn't ended and doesn't have a guaranteed win/draw/loss
    # from the endgame tablebase, evaluate the move normally
    score_diff = self.evaluate_attack(move)
    score_diff += self.evaluate_positions(color, move)
    score_diff += self.evaluate_castling(color, move)
    score_diff += self.evaluate_promotion(move)

    return prev_score + (score_diff * is_opponent_multiplier), False

chess.Board.evaluate_move = evaluate_move
del evaluate_move

#### chess.Board.evaluate_game_end
Finds if, after a given move, the chess game is over. If so, it also finds the winner associated with this game over.

###### <b><u>Arguments</u></b>
``move (chess.Move): ``  
The move that should be checked for a game over.

###### <b><u>Returns <i>(bool, chess.Color)</i></u></b>
- Whether or not the move lead to a game over.
- The winning color if a game over occured, None otherwise.

In [ ]:
def evaluate_game_end(self, move: chess.Move) -> (bool, chess.Color):
    self.push(move)
    game_ended = self.is_game_over()
    winner = None if self.outcome() is None else self.outcome().winner
    self.pop()

    return game_ended, winner

chess.Board.evaluate_game_end = evaluate_game_end
del evaluate_game_end

#### chess.Board.is_winning
Finds whether a certain color is winning, i.e. if they are in an advantageous position (but may not necessarily win the game, they just have the advantage). This is the case if their total board score, i.e. piece values plus positional scores, is greater than that of the opponent. If both players have an equal score, they are both considered to be winning (and will therefore try to avoid draws; this should make the game more interesting).

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of which the winning or losing position should be determined.

###### <b><u>Returns <i>(bool)</i></u></b>
Whether or not the given color is winning (see definition above).

In [ ]:
def is_winning(self, color: chess.Color) -> bool:
    score = 0
    for square in chess.SQUARES:
        piece = self.piece_at(square)
        if piece is None:
            continue
            
        piece_score_multiplier = 1 if piece.color == color else -1
        score += Globals.PIECE_VALUES[chess.piece_symbol(piece.piece_type)] \
                * piece_score_multiplier
        piece_square_table_key = self.get_piece_square_table_key(piece.piece_type)
        score += self.get_positional_score(color, piece_square_table_key, square) \
                * piece_score_multiplier

    return (score >= 0)

chess.Board.is_winning = is_winning
del is_winning

#### chess.Board.get_piece_square_table_key
Finds the dictionary key to index the piece square table of a piece type, which contains a positional score for each square on the chess board. Normally this key is simply the piece symbol of the piece type, but the king has a different piece square table during the end game.

###### <b><u>Arguments</u></b>
``piece_type (chess.PieceType):``  
The piece type to get the piece square table key for.

###### <b><u>Returns <i>(str)</i></u></b>
The piece square table key associated with the piece type.

In [ ]:
def get_piece_square_table_key(self, piece_type: chess.PieceType) -> str:
    piece_symbol = chess.piece_symbol(piece_type).lower()
    if piece_symbol == 'k':
        return 'ke' if self.ending else 'km'
    return piece_symbol

chess.Board.get_piece_square_table_key = get_piece_square_table_key
del get_piece_square_table_key

#### chess.Board.get_positional_score
Finds the positional score of a piece, which is retrieved from the matching piece square table.

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of the piece for which the positional score should be found.

``piece_square_table_key (str):``  
The dictionary key to index the piece square table of the piece's type.

``square (chess.Square):``  
The square (position) that the piece is located at.

###### <b><u>Returns <i>(int)</i></u></b>
The positional score.

In [ ]:
def get_positional_score(
    self, 
    color: chess.Color, 
    piece_square_table_key: str, 
    square: chess.Square
) -> int:

    hor_index = square % 8
    ver_index = 7 - square // 8 if color == chess.WHITE else square // 8
    piece_square_table = Globals.PIECE_SQUARE_TABLES[piece_square_table_key]
    piece_square_value = piece_square_table[ver_index][hor_index]
    return piece_square_value

chess.Board.get_positional_score = get_positional_score
del get_positional_score

#### chess.Board.evaluate_endgame
Calculates a score based on the information in the endgame tablebase. This based on the DTM (depth to mate) value, which lists the amount of half-moves until a forced mate. If this value is positive, the current side to move can win in that many moves. If the value is negative, the current side to move loses in that many moves (assuming perfect play by the opponent). If the value is zero, a draw is forced. The evaluation score is calculated as follows:
- If the DTM value is positive, the score is a very high positive number minus the DTM value (the fewer half-moves to win the better).
- If the DTM value is negative, the score is a very high negative number minus the DTM value (the more half-moves to lose the better, and since the DTM value is negative, the score is less negative for high DTM values).
- If the DTM value is zero, the score is positive if the player is losing and negative if the player is winning (see chess.Board.is_winning for a detailed explanation).

###### <b><u>Arguments</u></b>
``move (chess.Move):``  
The move that should be checked against the endgame tablebase.

``color (chess.Color):``  
The color that can make the move.

``endgame_tablebase (Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]):``  
The endgame tablebase attached to the game.

###### <b><u>Returns <i>(int)</i></u></b>
The evaluation score if the endgame tablebase contained DTM information for the current board state (after the move is made), otherwise None.

In [ ]:
def evaluate_endgame(
    self,
    move: chess.Move,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase,
            chess.gaviota.PythonTablebase]
) -> int:

    evaluation_score = 0
    self.push(move)

    try:
        dtm_score = endgame_tablebase.probe_dtm(self)
        if dtm_score > 0:
            evaluation_score = Globals.EVALUATION_SCORE_ENDGAME - dtm_score
        elif dtm_score < 0:
            evaluation_score = -Globals.EVALUATION_SCORE_ENDGAME - dtm_score
        else:
            evaluation_score = -Globals.EVALUATION_SCORE_DRAW if self.is_winning(color) \
                    else Globals.EVALUATION_SCORE_DRAW
    except KeyError:
        evaluation_score = None

    self.pop()
    return evaluation_score

chess.Board.evaluate_endgame = evaluate_endgame
del evaluate_endgame

#### chess.Board.evaluate_attack
Calculates the move score associated with an attack, which is the piece value of the opponent's attacked piece. If the move is not an attack, this score is 0.

###### <b><u>Arguments</u></b>
``move (chess.Move):``  
The move that should be checked for an attack.

###### <b><u>Returns <i>(int)</i></u></b>
The move score associated with the attack.

In [ ]:
def evaluate_attack(self, move: chess.Move) -> int:
    if self.is_en_passant(move):
        opposite_color = not self.piece_at(move.from_square).color
        to_piece = chess.Piece(chess.PAWN, opposite_color)
    else:
        to_piece = self.piece_at(move.to_square)
    # No need to check if the piece is of the opposite color
    # - attacking one's own color is not a legal move
    if to_piece is not None:
        return Globals.PIECE_VALUES[chess.piece_symbol(to_piece.piece_type)]
    return 0

chess.Board.evaluate_attack = evaluate_attack
del evaluate_attack

#### chess.Board.evaluate_positions
Calculates the move score associated with the "from" and "to" positions of a move. This is the "to" score minus the "from" score, each of which is retrieved from the piece square table of the corresponding piece.

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of the player who makes the move. 

``move (chess.Move):``  
The move of which the positions should be evaluated.

###### <b><u>Returns <i>(int)</i></u></b>
The move score associated with the positions.

In [ ]:
def evaluate_positions(self, color: chess.Color, move: chess.Move) -> int:
    from_piece = self.piece_at(move.from_square)
    from_piece_square_table_key = self.get_piece_square_table_key(from_piece.piece_type)
    to_piece_type = from_piece.piece_type if move.promotion is None else move.promotion
    to_piece_square_table_key = self.get_piece_square_table_key(to_piece_type)

    from_score = self.get_positional_score(color,
            from_piece_square_table_key, move.from_square)
    to_score = self.get_positional_score(color,
            to_piece_square_table_key, move.to_square)
    return to_score - from_score

chess.Board.evaluate_positions = evaluate_positions
del evaluate_positions

#### chess.Board.evaluate_castling
Calculates the move score associated with castling. This score is the positional score of the rook, as the king's positional score is already handled by evaluate_positions. If the move does not involve castling, this score is 0.

###### <b><u>Arguments</u></b>
``color (chess.Color):``   
The color of the player who makes the move.

``move (chess.Move):``  
The move for which castling should be evaluated.

###### <b><u>Returns <i>(int)</i></u></b>
The move score associated with the castling.

In [ ]:
def evaluate_castling(self, color: chess.Color, move: chess.Move) -> int:
    if not self.is_castling(move):
        return 0

    old_square, new_square = self.find_castling_rook_squares(color, move)
    old_score = self.get_positional_score(color, 'r', old_square)
    new_score = self.get_positional_score(color, 'r', new_square)
    return new_score - old_score

chess.Board.evaluate_castling = evaluate_castling
del evaluate_castling

#### chess.Board.find_castling_rook_squares
Finds the "from" and "to" square of the rook's movement during castling. This is necessary because the move's notation only contains the king's movement, which indirectly defines the rook's movement as well. <br>
The rook can only be in one of the four corners of the board prior to castling. Each corner also has a specified direction that the king moves in (1 if right and -1 if left). This function can therefore simply check for these four cases and place the rook behind the king in the correct direction.

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of the player who makes the move.

``move (chess.Move):``  
The castling move, describing the king's movement.

###### <b><u>Returns <i>(chess.Square, chess.Square)</i></u></b>
- The "from" square of the rook's movement.
- The "to" square of the rook's movement.

In [ ]:
def find_castling_rook_squares(
    self, 
    color: chess.Color, 
    move: chess.Move
) -> (chess.Square, chess.Square):

    king_move_dir = 1 if move.to_square > move.from_square else -1
    old_rook_square = (chess.A1 if king_move_dir == -1 else chess.H1) \
        if color == chess.WHITE \
        else (chess.A8 if king_move_dir == -1 else chess.H8)
    new_rook_square = move.to_square - king_move_dir

    return old_rook_square, new_rook_square

chess.Board.find_castling_rook_squares = find_castling_rook_squares
del find_castling_rook_squares

#### chess.Board.evaluate_promotion
Calculates the move score associated with a promotion, i.e. transforming a pawn into a queen, knight, rook is bishop. This score is defined as the piece value of the new piece minus the piece value of the pawn. If the move does not result in a promotion, this score is 0. The effect that a promotion has on the positional score is accounted for in evaluate_positions.

###### <b><u>Arguments</u></b>
``move (chess.Move):``  
The move for which a promotion should be evaluated.

###### <b><u>Returns <i>(int)</i></u></b>
The move score associated with the promotion.

In [ ]:
def evaluate_promotion(self, move: chess.Move) -> int:
    if move.promotion is None:
        return 0

    old_piece = self.piece_at(move.from_square)
    old_piece_symbol = chess.piece_symbol(old_piece.piece_type).lower()
    new_piece_symbol = chess.piece_symbol(move.promotion).lower()
    return Globals.PIECE_VALUES[new_piece_symbol] - Globals.PIECE_VALUES[old_piece_symbol]

chess.Board.evaluate_promotion = evaluate_promotion
del evaluate_promotion

#### chess.Board.get_state_string
Creates a string that represents the current state of a chess board, i.e. which pieces are at which position.

###### <b><u>Returns <i>(str)</i></u></b>
A string that represents the state of the board.

In [ ]:
def get_state_string(self) -> int:
    return chess.polyglot.zobrist_hash(self)

chess.Board.get_state_string = get_state_string
del get_state_string

#### chess.Board.check_and_set_ending
Finds if the game has entered its ending phase. This follows the definition of the Simplified Evaluation Function, see Michniewski, T. (2021). The ending phase is triggered when <b>either</b> of the following conditions is met:
- Both sides have no queens.
- Every side that has a queen has at most one minor piece and no other pieces (pawns are not counted).

###### <b><u>Side effects</u></b>
If the ending phase is triggered, the "ending" variable of the board is set to True, so that the ending phase does not have to be checked for again.

In [ ]:
def check_and_set_ending(self) -> None:
    if self.ending: return

    meets_ending_requirement = { chess.WHITE: False, chess.BLACK: False }
    for color in [ chess.WHITE, chess.BLACK ]:
        # Option 1: no queen
        if self.get_piece_count(color, chess.QUEEN) == 0:
            meets_ending_requirement[color] = True
            continue

        # Option 2: at most one minor piece and no other pieces (besides queen)
        # Most sources state that pawns aren't pieces, so these are not counted
        if self.get_piece_count(color, chess.BISHOP, chess.KNIGHT) <= 1 \
                and self.get_piece_count(color, chess.ROOK) == 0:
            meets_ending_requirement[color] = True

    if meets_ending_requirement[chess.WHITE] and meets_ending_requirement[chess.BLACK]:
        self.ending = True

chess.Board.check_and_set_ending = check_and_set_ending
del check_and_set_ending

#### chess.Board.get_piece_count
Finds how many pieces of certain types of a certain color are on the board. For example, this function can be used to find the combined total of white bishops and knights.

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of the pieces that should be counted.

``*piece_types (chess.PieceType):``  
The piece type(s) that should be counted.

###### <b><u>Returns <i>(int)</i></u></b>
The total amount of pieces of the given types and of the given color.

In [ ]:
def get_piece_count(self, color: chess.Color, *piece_types: chess.PieceType) -> int:
    return sum( len(self.pieces(piece_type, color)) for piece_type in piece_types )

chess.Board.get_piece_count = get_piece_count
del get_piece_count

#### chess.Board.get_search_result_if_finished
Checks if a search (minimax or alpha-beta) is finished. This is the case if either the game is over with the board state at the current search node, or if the maximum search depth has been reached. If the search is finished, it returns the resulting move score of this search.

###### <b><u>Arguments</u></b>
``color (chess.Color):``  
The color of the player that started the search (and should therefore make the move).  

``iteration (int):``  
The current depth of the search.  

``max_iterations (int):``  
The maximum depth of the search.

###### <b><u>Returns <i>(int)</i></u></b>
The move score if the search has ended, or None otherwise.

In [ ]:
def get_search_result_if_finished(
    self, 
    color: chess.Color, 
    iteration: int, 
    max_iterations: int, 
    eval_score: int
) -> int:

    # If the game has finished, return a positive score if we won,
    # or a negative score if we lost.
    # If the game ended in a draw, return a negative score if we are
    # in a winning position or a positive score otherwise.
    outcome = self.outcome()
    if outcome is not None:
        max_score = Globals.EVALUATION_SCORE_CHECKMATE - iteration
        min_score = -Globals.EVALUATION_SCORE_CHECKMATE + iteration
        if outcome.winner is None: # Draw
            return -Globals.EVALUATION_SCORE_DRAW if self.is_winning(color) \
                    else Globals.EVALUATION_SCORE_DRAW
        if outcome.winner == color: # Win
            return max_score
        return min_score # Loss
        
    # If the search has hit its max depth (max_iterations), return the
    # current evaluation score for the AI.
    if iteration >= max_iterations:
        return eval_score

    # The search has not ended, so no result.
    return None

chess.Board.get_search_result_if_finished = get_search_result_if_finished
del get_search_result_if_finished

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>